# Bibliotecas

In [112]:
import pandas as pd
from pathlib import Path
import os

# Tratar bases

In [134]:
# Diretório base onde o arquivo CSV está localizado
base_diretório = Path(os.getcwd())
diretório_princapl = base_diretório.parent
caminho = diretório_princapl / 'Base'

# Definir os caminhos para cada criptomoeda
arquivo_ibovespa = caminho / 'Dados Históricos - Ibovespa.csv'

In [135]:
def tratar_dados_cripto(caminho_csv):
    # Força leitura como string para evitar perda de zeros
    acao = pd.read_csv(caminho_csv, dtype=str)

    # Converter a coluna de data
    acao['Data'] = pd.to_datetime(acao['Data'], format='%d.%m.%Y')

    # Corrigir colunas numéricas
    colunas_dinheiro = ['Último', 'Abertura', 'Máxima', 'Mínima']
    for coluna in colunas_dinheiro:
        acao[coluna] = (
            acao[coluna]
            .str.replace('.', '', regex=False)  # Remove separador de milhar
            .str.replace(',', '', regex=False)  # Remove separador decimal
            .astype(int)
        )

    # Corrigir variação percentual
    acao['Var%'] = (
        acao['Var%']
        .str.replace('%', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float) / 100
    )
    acao['Var%'] = acao['Var%'].round(4)
    acao = acao.rename(columns={'Var%': 'Variacao (%)'})

    # Remover a coluna de volume
    acao = acao.drop(columns=['Vol.'])

    # Ordenar por data
    acao = acao.sort_values(by='Data')

    return acao

# Aplicar função e salvar resultado
df_ibovespa = tratar_dados_cripto(arquivo_ibovespa)
df_ibovespa.to_csv("Base Ibovespa Tratada.csv", index=False)


In [106]:
# Função para tratar os dados
def tratar_dados_cripto(caminho_csv):
    # Ler o arquivo CSV
    acao = pd.read_csv(caminho_csv)

    # Formatar a coluna de data
    acao['Data'] = pd.to_datetime(acao['Data'], format='%d.%m.%Y')

    # Converter a coluna 'Var%' para float decimal
    acao['Var%'] = acao['Var%'].str.replace('%', '', regex=False).str.replace(',', '.', regex=False).astype(float) / 100
    acao['Var%'] = acao['Var%'].round(4)

    acao = acao.drop(columns=['Vol.'])

    # Renomear a coluna 'Var%'
    acao = acao.rename(columns={'Var%': 'Variacao (%)'})

    # Ordenar os dados pela data
    acao = acao.sort_values(by='Data', ascending=True)

    return acao

# Aplicar a função
df_ibovespa = tratar_dados_cripto(arquivo_ibovespa)


df_ibovespa.to_csv("Base Ibovespa Tratada.csv", index=False)


In [133]:
df_ibovespa.dtypes

Data            datetime64[ns]
Último                   int64
Abertura                 int64
Máxima                   int64
Mínima                   int64
Variacao (%)           float64
dtype: object